In [8]:
import pandas as pd


In [12]:

arquivos = ["DadosBase2023.csv", "DadosBase2024.csv", "SH.csv", "NomesMunicipios.csv"]

url_base = "https://raw.githubusercontent.com/Lucasdz-7/Projeto-Integrador-1/f81f64f9a0324de118ead77d63aba9e33bb98e32/Projeto/"

dataframes = {}

for arquivo in arquivos:
    url = url_base + arquivo
    print(f"Baixando {arquivo} de {url}...")

    try:
        df = pd.read_csv(url, sep=";", encoding="ISO-8859-1")
        dataframes[arquivo] = df
        print(f"{arquivo} carregado com sucesso!")
    except Exception as e:
        print(f"Erro ao baixar {arquivo}: {e}")

if arquivos:
    primeiro_arquivo = arquivos[0]
    print(f"\nVisualizando {primeiro_arquivo}:")
    print(dataframes[primeiro_arquivo].head())


Baixando DadosBase2023.csv de https://raw.githubusercontent.com/Lucasdz-7/Projeto-Integrador-1/f81f64f9a0324de118ead77d63aba9e33bb98e32/Projeto/DadosBase2023.csv...
DadosBase2023.csv carregado com sucesso!
Baixando DadosBase2024.csv de https://raw.githubusercontent.com/Lucasdz-7/Projeto-Integrador-1/f81f64f9a0324de118ead77d63aba9e33bb98e32/Projeto/DadosBase2024.csv...
DadosBase2024.csv carregado com sucesso!
Baixando SH.csv de https://raw.githubusercontent.com/Lucasdz-7/Projeto-Integrador-1/f81f64f9a0324de118ead77d63aba9e33bb98e32/Projeto/SH.csv...
SH.csv carregado com sucesso!
Baixando NomesMunicipios.csv de https://raw.githubusercontent.com/Lucasdz-7/Projeto-Integrador-1/f81f64f9a0324de118ead77d63aba9e33bb98e32/Projeto/NomesMunicipios.csv...
NomesMunicipios.csv carregado com sucesso!

Visualizando DadosBase2023.csv:
   CO_ANO  CO_MES   SH4  CO_PAIS SG_UF_MUN   CO_MUN  KG_LIQUIDO  VL_FOB
0    2023      10  2209      127        BA  2919553          36      53
1    2023      11  4009   